In [ ]:
# !pip uninstall mediapipe opencv-python==4.5.5.64
!pip uninstall mediapipe opencv-python==0.9.3.0
# !pip uninstall opencv-python
# !pip uninstall opencv-contrib-python
# !pip uninstall opencv-contrib-python-headless
# !pip uninstall -U tensorflow-estimator==2.6.0
# !pip uninstall tensorflow --upgrade

In [ ]:
!pip install mediapipe
# !pip install mediapipe

In [68]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import pandas as pd
import math
from statistics import mean
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [59]:
# Calculate angle
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


# Calculate distance
def calculate_distance(a,b):
    x1, y1 = a
    x2, y2 = b
    
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    return distance


# Calculate velocity
def calculate_velocity(distance, time):
    velocity = distance / time
    
    return velocity


In [54]:
# Create CSV file
csvfile = open('bicep_curl.csv', 'w', newline='')
csvwriter = csv.writer(csvfile)
# csvwriter.writerow(['shoulder_x', 'shoulder_y', 'elbow_x', 'elbow_y', 'wrist_x', 'wrist_y'])
csvwriter.writerow(['shoulder', 'elbow', 'wrist'])

22

In [77]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "contract"
            if angle < 30 and stage =='contract':
                stage="extend"
                counter +=1
                print(counter)
            
            # Calculate rep score
                # Log all frames to temp_frames
                # Call function(rep, temp_frames)
                    # Elbow inline with shoulder for all frames? Else: Elbow = Bad, Break
                    # Velocity below threshold for all frames? Else: Velocity = Bad, Break
                    # Range within threshold for all frames? Range: Rep = Bad, Break
                # Log elbow, velocity, range feedback to feedback.csv 
                # Return "Bad", score -= 1 && clear temp_frames
                
            
            # Save to CSV
#             csvwriter.writerow([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
#                                 [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
#                                 [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]])
            csvwriter.writerow([shoulder, elbow, wrist])

        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    csvfile.close()
    cap.release()
    cv2.destroyAllWindows()

# Read CSV file
df = pd.read_csv('bicep_curl.csv')
print(df)

                                     shoulder  \
0    [0.5704053640365601, 0.7455049157142639]   
1    [0.5700571537017822, 0.7455533146858215]   
2    [0.5700896978378296, 0.7466236352920532]   
3    [0.5700625777244568, 0.7470922470092773]   
4    [0.5703216195106506, 0.7481060028076172]   
..                                        ...   
128  [0.6685722470283508, 0.8541804552078247]   
129  [0.6728945374488831, 0.8582033514976501]   
130  [0.6804550886154175, 0.8613144159317017]   
131  [0.6878982782363892, 0.8714280724525452]   
132  [0.6887826919555664, 0.8671265244483948]   

                                        elbow  \
0    [0.6689730882644653, 0.8512599468231201]   
1    [0.6645397543907166, 0.8511189222335815]   
2     [0.6618248820304871, 0.851199209690094]   
3    [0.6597956418991089, 0.8506298065185547]   
4    [0.6581414937973022, 0.8510599732398987]   
..                                        ...   
128  [0.8301202654838562, 1.1395286321640015]   
129  [0.86787033081

In [ ]:
# Read CSV file
df = pd.read_csv('bicep_curl.csv')
print(df)

In [78]:
# Define the ideal form criteria and the scoring system
ELBOW_POSITION_THRESHOLD = 0.2  # Maximum allowed distance between elbow and shoulder keypoints
FULL_RANGE_OF_MOTION_ANGLE = 90  # Ideal angle between shoulder, elbow, and wrist keypoints
SMOOTHNESS_THRESHOLD = 10  # Maximum allowed change in velocity between wrist keypoints

PERFECT_FORM_SCORE = 10  # Maximum score for perfect form
ELBOW_POSITION_PENALTY = 1  # Penalty per unit distance above threshold
RANGE_OF_MOTION_PENALTY = 1  # Penalty per unit angle deviation from ideal
SMOOTHNESS_PENALTY = 1  # Penalty per unit velocity change above threshold

# Calculate the measurements for each repetition
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

elbow_distance = calculate_distance(elbow, shoulder)
rom_angle = calculate_angle(shoulder, elbow, wrist)
# velocity = calculate_velocity(wrist_history)

# Compare the measurements to the ideal form criteria and assign a score
elbow_penalty = max(0, (elbow_distance - ELBOW_POSITION_THRESHOLD) / ELBOW_POSITION_PENALTY)
rom_penalty = max(0, (abs(rom_angle - FULL_RANGE_OF_MOTION_ANGLE)) / RANGE_OF_MOTION_PENALTY)
# smoothness_penalty = max(0, (velocity - SMOOTHNESS_THRESHOLD) / SMOOTHNESS_PENALTY)

# form_score = PERFECT_FORM_SCORE - elbow_penalty - rom_penalty #- smoothness_penalty
form_score = mean((elbow_penalty, rom_penalty)) #- smoothness_penalty)

# Keep a running total of the scores for each repetition
total_score += form_score

# Report the average score at the end of the exercise
num_repetitions = counter
# average_score = total_score / num_repetitions
average_score = total_score

print(f"Average score for {num_repetitions} = {average_score}")


Average score for 0 = 159.24762928464557
